In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive/')

# !ls

In [2]:
import os

home_dir = '/content/gdrive/My Drive/AChina' 
home_dir = '/hy-tmp'
os.chdir(home_dir)
!pwd

from glob import glob
from tqdm import tqdm

import numpy as np
import pandas as pd

/hy-tmp


In [3]:
import os
import warnings

warnings.filterwarnings("ignore")  # avoid printing out absolute paths

# os.chdir("../../..")

In [4]:
# !pip install torch torchvision
# !pip install pytorch_lightning
# !pip install pytorch_forecasting

# !pip install torch -f https://download.pytorch.org/whl/torch_stable.html
# !pip install pytorch-forecasting

!pip install scipy
!pip install torch pytorch-lightning pytorch_forecasting

import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAPE, SMAPE, PoissonLoss, QuantileLoss
# from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters


Looking in indexes: https://mirrors.aliyun.com/pypi/simple
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.


In [5]:
data = pd.read_csv('corn_china_pandas.csv')  # encoding= 'unicode_escape')

data[0:15]

,Unnamed: 0,years,yield,county,bands,time_idx,bin0,bin1,bin2,bin3,...,bin502,bin503,bin504,bin505,bin506,bin507,bin508,bin509,bin510,bin511
0,0,2003,0.61295,0,band_0,0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,2003,0.61295,0,band_1,0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,2003,0.61295,0,band_2,0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,2003,0.61295,0,band_3,0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,4,2003,0.61295,0,band_4,0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,5,2003,0.61295,0,band_5,0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,6,2003,0.61295,0,band_6,0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,7,2003,0.61295,0,band_7,0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,8,2003,0.61295,0,band_8,0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,9,2003,0.61295,0,band_0,1,0.0,0.0,0.0,0.0,...,0.001077,0.000833,0.001126,0.000098,0.000784,0.000881,0.000979,0.000441,0.000294,0.000196


In [6]:
# years = [x for x in range(2003, 2019)]

# data.rename(columns={'time_idx' : 'time'}, inplace=True)  

# # data[5:15]  
# data.insert(1, "time_idx", data['years'])  
# # df = data.assign(time_dx = data.time * 10)

# time_idx = 0
# for year in years:
#     data['time_idx'] = data['time_idx'].replace([year], time_idx)
#     time_idx = time_idx + 1
    
# data['years'] = data['years'].astype(str)
# data['county'] = data['county'].astype(str)
# data['time'] = data['time'].astype(str)

# dff = data[ data['years'] == '2018' ]
# dff

In [7]:
data['years'] = data['years'].astype(str)
data['county'] = data['county'].astype(str)
data['time_idx'] = data['time_idx'].astype(np.int64)
data.head()

,Unnamed: 0,years,yield,county,bands,time_idx,bin0,bin1,bin2,bin3,...,bin502,bin503,bin504,bin505,bin506,bin507,bin508,bin509,bin510,bin511
0,0,2003,0.61295,0,band_0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2003,0.61295,0,band_1,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2003,0.61295,0,band_2,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2003,0.61295,0,band_3,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2003,0.61295,0,band_4,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
data.describe()

,Unnamed: 0,yield,time_idx,bin0,bin1,bin2,bin3,bin4,bin5,bin6,...,bin502,bin503,bin504,bin505,bin506,bin507,bin508,bin509,bin510,bin511
count,91872.000000,91872.000000,91872.000000,91872.000000,91872.000000,91872.000000,91872.000000,91872.000000,91872.000000,91872.000000,...,91872.000000,91872.000000,91872.000000,91872.000000,91872.000000,91872.000000,91872.000000,91872.000000,91872.000000,91872.000000
mean,4175.500000,0.517953,15.500000,0.000733,0.000152,0.000182,0.000171,0.000206,0.000194,0.000231,...,0.000247,0.000256,0.000257,0.000240,0.000252,0.000254,0.000228,0.000244,0.000231,0.000254
std,2411.027829,0.106652,9.233143,0.003606,0.000685,0.000838,0.000762,0.000930,0.000845,0.001023,...,0.000711,0.000745,0.000783,0.000701,0.003375,0.001853,0.000673,0.000899,0.000685,0.000792
min,0.000000,0.276526,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2087.750000,0.438383,7.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4175.500000,0.518895,15.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6263.250000,0.574336,23.250000,0.000059,0.000020,0.000023,0.000023,0.000026,0.000026,0.000028,...,0.000148,0.000156,0.000151,0.000142,0.000141,0.000144,0.000127,0.000135,0.000129,0.000141
max,8351.000000,0.855448,31.000000,0.111111,0.025479,0.027569,0.024157,0.028934,0.022859,0.028043,...,0.026096,0.040541,0.054054,0.024242,1.000000,0.500000,0.021858,0.157895,0.032663,0.066667


In [9]:
import torch
from torch import nn

class FullyConnectedModule(nn.Module):
    def __init__(self, input_size: int, output_size: int, hidden_size: int, n_hidden_layers: int):
        super().__init__()

        # input layer
        module_list = [nn.Linear(input_size, hidden_size), nn.ReLU()]
        # hidden layers
        for _ in range(n_hidden_layers):
            module_list.extend([nn.Linear(hidden_size, hidden_size), nn.ReLU()])
        # output layer
        module_list.append(nn.Linear(hidden_size, output_size))

        self.sequential = nn.Sequential(*module_list)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x of shape: batch_size x n_timesteps_in
        # output of shape batch_size x n_timesteps_out
        return self.sequential(x)


# test that network works as intended
network = FullyConnectedModule(input_size=5, output_size=2, hidden_size=10, n_hidden_layers=2)
x = torch.rand(20, 5)
network(x).shape

torch.Size([20, 2])

In [10]:
from typing import Dict, List, Tuple
from pytorch_forecasting.models.nn import MultiEmbedding
from pytorch_forecasting.models import BaseModelWithCovariates

class FullyConnectedModelWithCovariates(BaseModelWithCovariates):
    def __init__(
        self,
        input_size: int,
        output_size: int,
        hidden_size: int,
        n_hidden_layers: int,
        x_reals: List[str],
        x_categoricals: List[str],
        embedding_sizes: Dict[str, Tuple[int, int]],
        embedding_labels: Dict[str, List[str]],
        static_categoricals: List[str],
        static_reals: List[str],
        time_varying_categoricals_encoder: List[str],
        time_varying_categoricals_decoder: List[str],
        time_varying_reals_encoder: List[str],
        time_varying_reals_decoder: List[str],
        embedding_paddings: List[str],
        categorical_groups: Dict[str, List[str]],
        **kwargs,
    ):
        # saves arguments in signature to `.hparams` attribute, mandatory call - do not skip this
        self.save_hyperparameters()
        # pass additional arguments to BaseModel.__init__, mandatory call - do not skip this
        super().__init__(**kwargs)

        # create embedder - can be fed with x["encoder_cat"] or x["decoder_cat"] and will return
        # dictionary of category names mapped to embeddings
        self.input_embeddings = MultiEmbedding(
            embedding_sizes=self.hparams.embedding_sizes,
            categorical_groups=self.hparams.categorical_groups,
            embedding_paddings=self.hparams.embedding_paddings,
            x_categoricals=self.hparams.x_categoricals,
            max_embedding_size=self.hparams.hidden_size,
        )
        
        # calculate the size of all concatenated embeddings + continous variables
        n_features = sum(
            embedding_size for classes_size, embedding_size in self.hparams.embedding_sizes.values()
        ) + len(self.reals)

        # create network that will be fed with continious variables and embeddings
        self.network = FullyConnectedModule(
            input_size=self.hparams.input_size * n_features,
            output_size=self.hparams.output_size,
            hidden_size=self.hparams.hidden_size,
            n_hidden_layers=self.hparams.n_hidden_layers,
        )

    def forward(self, x: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        # x is a batch generated based on the TimeSeriesDataset
        batch_size = x["encoder_lengths"].size(0)
        embeddings = self.input_embeddings(x["encoder_cat"])  # returns dictionary with embedding tensors
        network_input = torch.cat(
            [x["encoder_cont"]]
            + [
                emb
                for name, emb in embeddings.items()
                if name in self.encoder_variables or name in self.static_variables
            ],
            dim=-1,
        )
        prediction = self.network(network_input.view(batch_size, -1))

        # rescale predictions into target space
        prediction = self.transform_output(prediction, target_scale=x["target_scale"])

        # We need to return a dictionary that at least contains the prediction.
        # The parameter can be directly forwarded from the input.
        # The conversion to a named tuple can be directly achieved with the `to_network_output` function.
        return self.to_network_output(prediction=prediction)
    
    @classmethod
    def from_dataset(cls, dataset: TimeSeriesDataSet, **kwargs):
        new_kwargs = {
            "output_size": dataset.max_prediction_length,
            "input_size": dataset.max_encoder_length,
        }
        new_kwargs.update(kwargs)  # use to pass real hyperparameters and override defaults set by dataset
        # example for dataset validation
        assert dataset.max_prediction_length == dataset.min_prediction_length, "Decoder only supports a fixed length"
        assert dataset.min_encoder_length == dataset.max_encoder_length, "Encoder only supports a fixed length"

        return super().from_dataset(dataset, **new_kwargs)

In [11]:
from pytorch_lightning.utilities.model_summary import summarize
# create the dataset from the pandas dataframe
train_data = data[ data["years"] != "2018" ]
valid_data = data[ data["years"] == "2018" ]

train_dataset_with_covariates = TimeSeriesDataSet(
    train_data,
    group_ids=["years", "county", "bands"],
    target="yield",
    time_idx="time_idx",
    min_encoder_length=5,
    max_encoder_length=5,
    min_prediction_length=1,
    max_prediction_length=1,
    time_varying_unknown_reals=["yield"],
    # time_varying_known_reals=["real_covariate"],
    time_varying_known_categoricals=["county"],
    static_categoricals=["years", "county"],
)

valid_dataset_with_covariates = TimeSeriesDataSet(
    valid_data,
    group_ids=["years", "county", "bands"],
    target="yield",
    time_idx="time_idx",
    min_encoder_length=5,
    max_encoder_length=5,
    min_prediction_length=1,
    max_prediction_length=1,
    time_varying_unknown_reals=["yield"],
    # time_varying_known_reals=["real_covariate"],
    time_varying_known_categoricals=["county"],
    static_categoricals=["years", "county"],
)

model = FullyConnectedModelWithCovariates.from_dataset(train_dataset_with_covariates, hidden_size=10, n_hidden_layers=2)
summarize(model,max_depth=-1)  # print model summary
model.hparams

"categorical_groups":                {}
"embedding_labels":                  {'years': {'2003': 0, '2004': 1, '2005': 2, '2006': 3, '2007': 4, '2008': 5, '2009': 6, '2010': 7, '2011': 8, '2012': 9, '2013': 10, '2014': 11, '2015': 12, '2016': 13, '2017': 14}, 'county': {'0': 0, '1': 1, '11': 2, '12': 3, '13': 4, '14': 5, '15': 6, '16': 7, '17': 8, '19': 9, '2': 10, '21': 11, '23': 12, '24': 13, '26': 14, '29': 15, '3': 16, '5': 17, '8': 18, '9': 19}}
"embedding_paddings":                []
"embedding_sizes":                   {'years': (15, 7), 'county': (20, 9)}
"hidden_size":                       10
"input_size":                        5
"learning_rate":                     0.001
"log_gradient_flow":                 False
"log_interval":                      -1
"log_val_interval":                  -1
"logging_metrics":                   ModuleList()
"loss":                              SMAPE()
"monotone_constaints":               {}
"n_hidden_layers":                   2
"optimizer":

In [12]:
model = FullyConnectedModelWithCovariates.from_dataset(train_dataset_with_covariates, hidden_size=10, n_hidden_layers=2)

# convert datasets to dataloaders for training
batch_size = 128
train_dataloader = train_dataset_with_covariates.to_dataloader(train=True,  batch_size=batch_size, num_workers=2)
valid_dataloader = valid_dataset_with_covariates.to_dataloader(train=False, batch_size=batch_size, num_workers=2)

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger('/tf_logs', name="FCwC_model")

# trainer = Trainer(gpus=1, max_epochs=100, limit_train_batches=2606, logger=logger)
trainer = Trainer(gpus=1, logger=logger)
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=valid_dataloader)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type                 | Params
----------------------------------------------------------
0 | loss             | SMAPE                | 0     
1 | logging_metrics  | ModuleList           | 0     
2 | input_embeddings | MultiEmbedding       | 285   
3 | network          | FullyConnectedModule | 1.1 K 
----------------------------------------------------------
1.4 K     Trainable params
0         Non-trainable params
1.4 K     Total params
0.006     Total estimated model params size 

Epoch 0:  94%|█████████▍| 569/606 [00:19<00:01, 29.05it/s, loss=0.0322, v_num=14, train_loss_step=0.0297]
Validation: 0it [00:00, ?it/s]
Epoch 1:  94%|█████████▍| 569/606 [00:19<00:01, 28.69it/s, loss=0.0076, v_num=14, train_loss_step=0.009, val_loss=0.0416, train_loss_epoch=0.116]   
Validation: 0it [00:00, ?it/s]
Epoch 2:  94%|█████████▍| 569/606 [00:20<00:01, 27.71it/s, loss=0.00288, v_num=14, train_loss_step=0.0025, val_loss=0.00839, train_loss_epoch=0.0154] 
Validation: 0it [00:00, ?it/s]
Epoch 3:  94%|█████████▍| 569/606 [00:20<00:01, 27.45it/s, loss=0.00185, v_num=14, train_loss_step=0.00184, val_loss=0.00539, train_loss_epoch=0.00434]
Validation: 0it [00:00, ?it/s]
Epoch 4:  94%|█████████▍| 569/606 [00:21<00:01, 27.01it/s, loss=0.00162, v_num=14, train_loss_step=0.00156, val_loss=0.00425, train_loss_epoch=0.00224]
Validation: 0it [00:00, ?it/s]
Epoch 5:  94%|█████████▍| 569/606 [00:20<00:01, 27.85it/s, loss=0.0013, v_num=14, train_loss_step=0.0011, val_loss=0.00432, train_loss_

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Epoch 65:  94%|█████████▍| 569/606 [00:20<00:01, 27.93it/s, loss=0.000338, v_num=14, train_loss_step=0.000414, val_loss=0.00373, train_loss_epoch=0.000576]
Validation: 0it [00:00, ?it/s]
Epoch 66:  73%|███████▎  | 443/606 [00:15<00:05, 27.87it/s, loss=0.00031, v_num=14, train_loss_step=0.000405, val_loss=0.00364, train_loss_epoch=0.000385] 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Epoch 71:  94%|█████████▍| 569/606 [00:20<00:01, 27.48it/s, loss=0.000457, v_num=14, train_loss_step=0.000723, val_loss=0.00355, train_loss_epoch=0.000417]
Validation: 0it [00:00, ?it/s]
Epoch 72:  34%|███▍      | 207/606 [00:07<00:14, 26.78it/s, loss=0.000926, v_num=14, train_loss_step=0.000535, val_loss=0.00378, train_loss_epoch=0.000406]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Epoch 73:  94%|█████████▍| 569/606 [00:20<00:01, 27.63it/s, loss=0.000372, v_num=14, train_loss_step=0.000549, val_loss=0.00354, train_loss_epoch=0.000635]
Validation: 0it [00:00, ?it/s]
Epoch 86:  94%|█████████▍| 569/606 [00:20<00:01, 27.56it/s, loss=0.000372, v_num=14, train_loss_step=0.000325, val_loss=0.00378, train_loss_epoch=0.000391]
Validation: 0it [00:00, ?it/s]
Epoch 87:  94%|█████████▍| 569/606 [00:21<00:01, 26.99it/s, loss=0.000393, v_num=14, train_loss_step=0.000208, val_loss=0.00359, train_loss_epoch=0.000415]
Validation: 0it [00:00, ?it/s]
Epoch 88:  94%|█████████▍| 569/606 [00:20<00:01, 28.22it/s, loss=0.000794, v_num=14, train_loss_step=0.00033, val_loss=0.00375, train_loss_epoch=0.000388] 
Validation: 0it [00:00, ?it/s]
Epoch 88:  97%|█████████▋| 587/606 [00:21<00:00, 27.89it/s, loss=0.000794, v_num=14, train_loss_step=0.00033, val_loss=0.00375, train_loss_epoch=0.000388]


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Epoch 92:  94%|█████████▍| 569/606 [00:19<00:01, 28.81it/s, loss=0.000378, v_num=14, train_loss_step=0.000358, val_loss=0.00356, train_loss_epoch=0.000362]
Validation: 0it [00:00, ?it/s]
Epoch 93:  94%|█████████▍| 569/606 [00:19<00:01, 28.73it/s, loss=0.000386, v_num=14, train_loss_step=0.000327, val_loss=0.0035, train_loss_epoch=0.000468]
Validation: 0it [00:00, ?it/s]
Epoch 94:  94%|█████████▍| 569/606 [00:19<00:01, 28.97it/s, loss=0.000283, v_num=14, train_loss_step=0.000203, val_loss=0.00368, train_loss_epoch=0.000388]
Validation: 0it [00:00, ?it/s]
Epoch 95:  94%|█████████▍| 569/606 [00:20<00:01, 27.64it/s, loss=0.00033, v_num=14, train_loss_step=0.000283, val_loss=0.00363, train_loss_epoch=0.000376] 
Validation: 0it [00:00, ?it/s]
Epoch 96:  94%|█████████▍| 569/606 [00:20<00:01, 28.28it/s, loss=0.000294, v_num=14, train_loss_step=0.000258, val_loss=0.00352, train_loss_epoch=0.000375]
Validation: 0it [00:00, ?it/s]
Epoch 97:  94%|█████████▍| 569/606 [00:20<00:01, 28.10it/s, loss=0

In [ ]:
fn

In [ ]:
max_prediction_length = 3
max_encoder_length = 8
training_cutoff = data["time_idx"].max() - max_prediction_length

# print(training_cutoff)
# x = data[lambda x: x.time_idx <= training_cutoff]
# print()

bins_name = list(["yield"])
for bin in range(0, 512):
  bins_name.append(f'bin{bin}')

print(bins_name)

training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx = "time_idx",
    target = "yield",
    group_ids = ["county", "bands", "time"],
    min_encoder_length = max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length = max_encoder_length,
    min_prediction_length = 1,
    max_prediction_length = max_prediction_length,
    # static_categoricals = ["county"],
    # static_reals=["avg_population_2017", "avg_yearly_household_income_2017"],
    # time_varying_known_categoricals=["special_days", "month"],
    # variable_groups={"special_days": special_days},  # group of categorical variables can be treated as one variable
    # time_varying_known_reals=["time_idx", "price_regular", "discount_in_percent"],
    # time_varying_unknown_categoricals=[],
    time_varying_unknown_reals = bins_name,
    allow_missing_timesteps = True,
    # target_normalizer=GroupNormalizer(
    #     groups=["agency", "sku"], transformation="softplus"
    # ),  # use softplus and normalize by group
    # add_relative_time_idx = True,
    # add_target_scales = True,
    # add_encoder_length = True,
)

# create validation set (predict=True) which means to predict the last max_prediction_length points in time
# for each series
validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)

# create dataloaders for model
batch_size = 1  # set this between 32 to 128
train_dataloader = training.to_dataloader(train =  True, batch_size = batch_size, num_workers = 0)
val_dataloader = validation.to_dataloader(train = False, batch_size = batch_size, num_workers = 0)


In [ ]:
# calculate baseline mean absolute error, i.e. predict next value as the last available value from the history
actuals = torch.cat([y for x, (y, weight) in iter(val_dataloader)])
baseline_predictions = Baseline().predict(val_dataloader)
(actuals - baseline_predictions).abs().mean().item()

In [ ]:
# configure network and trainer
# pl.seed_everything(42)
# trainer = pl.Trainer(
#     gpus=0,
#     # clipping gradients is a hyperparameter and important to prevent divergance
#     # of the gradient for recurrent neural networks
#     gradient_clip_val=0.1,
#     auto_lr_find = True,
# )


# tft = TemporalFusionTransformer.from_dataset(
#     training,
#     # not meaningful for finding the learning rate but otherwise very important
#     learning_rate=0.03,
#     hidden_size=16,  # most important hyperparameter apart from learning rate
#     # number of attention heads. Set to up to 4 for large datasets
#     attention_head_size=1,
#     dropout=0.1,  # between 0.1 and 0.3 are good values
#     hidden_continuous_size=8,  # set to <= hidden_size
#     output_size=1,  # 7 quantiles by default
#     loss=QuantileLoss(),
#     # reduce learning rate if no improvement in validation loss after x epochs
#     reduce_on_plateau_patience=4,
# )
# print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

In [ ]:
# # find optimal learning rate
# res = trainer.tuner.lr_find(
#     tft,
#     train_dataloaders=train_dataloader,
#     val_dataloaders=val_dataloader,
#     max_lr=10.0,
#     min_lr=1e-6,
# )

# # res = trainer.tuner.lr_find(tft, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader,)

# print(f"suggested learning rate: {res.suggestion()}")
# fig = res.plot(show=True, suggest=True)
# fig.show()

In [ ]:
# configure network and trainer
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

trainer = pl.Trainer(
    max_epochs=300,
    gpus=0,
    enable_model_summary=True,
    gradient_clip_val=0.1,
    limit_train_batches=30,  # coment in for training, running valiation every 30 batches
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=1,  # 7 quantiles by default
    loss=MAPE(),  #QuantileLoss(), #MAPE(),
    log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

In [ ]:
import tensorflow as tf 
import tensorboard as tb 
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

In [ ]:
# fit network
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

In [ ]:
# load the best model according to the validation loss
# (given that we use early stopping, this is not necessarily the last epoch)
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

In [ ]:
# calcualte mean absolute error on validation set
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
predictions = best_tft.predict(val_dataloader)
(actuals - predictions).abs().mean()

In [ ]:
# raw predictions are a dictionary from which all kind of information including quantiles can be extracted
raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)

In [ ]:
for idx in range(10):  # plot 10 examples
    best_tft.plot_prediction(x, raw_predictions, idx=idx, add_loss_to_title=True);

In [ ]:
fn

In [ ]:
import pickle

from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

# create study
study = optimize_hyperparameters(
    train_dataloader,
    val_dataloader,
    model_path="optuna_test",
    n_trials=200,
    max_epochs=50,
    gradient_clip_val_range=(0.01, 1.0),
    hidden_size_range=(8, 128),
    hidden_continuous_size_range=(8, 128),
    attention_head_size_range=(1, 4),
    learning_rate_range=(0.001, 0.1),
    dropout_range=(0.1, 0.3),
    trainer_kwargs=dict(limit_train_batches=30),
    reduce_on_plateau_patience=4,
    use_learning_rate_finder=False,  # use Optuna to find ideal learning rate or use in-built learning rate finder
)

# save study results - also we can resume tuning at a later point in time
with open("test_study.pkl", "wb") as fout:
    pickle.dump(study, fout)

# show best hyperparameters
print(study.best_trial.params)